In [ ]:
!python -m spacy download pt_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 32.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import spacy
import numpy as np
import pandas as pd
from gensim.models import KeyedVectors

In [ ]:
import numpy as np
from gensim.models import KeyedVectors
#Caregando modelos
w2v_modelo_cbow = KeyedVectors.load_word2vec_format('/content/drive/MyDrive/Alura/ML_Avançada/NLP/modelo_cbow.txt')
w2v_modelo_sg = KeyedVectors.load_word2vec_format('/content/drive/MyDrive/Alura/ML_Avançada/NLP/modelo_skipgram.txt')


In [ ]:
artigo_treino = pd.read_csv('/content/drive/MyDrive/Alura/ML_Avançada/NLP/treino.csv')
artigo_teste = pd.read_csv('/content/drive/MyDrive/Alura/ML_Avançada/NLP/teste.csv')

##Função tokenizador
- doc = nlp(texto):

- A variável doc contém o texto processado pelo modelo spaCy.

- tokens_validos = []:
Esta lista armazenará os tokens que atendem aos critérios de validação.
for token in doc:

- Este loop percorre cada token no documento processado.

- e_valido = not token.is_stop and token.is_alpha:

- token.is_stop verifica se o token é uma stop word (palavra muito comum que geralmente é ignorada no processamento de linguagem natural).

- token.is_alpha verifica se o token é composto apenas por caracteres alfabéticos (ou seja, não contém números ou caracteres especiais).

- e_valido será True apenas se o token não for uma stop word e for composto apenas por letras.

- if e_valido: tokens_validos.append(token.text.lower()):
Se o token for válido, ele é convertido para minúsculas e adicionado à lista tokens_validos.
return tokens_validos:

- A função retorna a lista de tokens válidos.

In [ ]:
nlp = spacy.load("pt_core_news_sm", disable=["paser", "ner", "tagger", "textcat"])

def tokenizador(texto):

    doc = nlp(texto)
    tokens_validos = []
    for token in doc:
        e_valido = not token.is_stop and token.is_alpha
        if e_valido:
            tokens_validos.append(token.text.lower())


    return  tokens_validos

texto = "Rio de Janeiro 1231231 ***** @#$ é uma cidade maravilhosa!"
tokens = tokenizador(texto)
print(tokens)

['rio', 'janeiro', 'cidade', 'maravilhosa']


##Função combinação de vetores por soma

- Aqui, a função cria um vetor de zeros de dimensão (1, 300). Esse vetor será usado para acumular a soma dos vetores das palavras. O tamanho 300 é escolhido assumindo que os vetores no modelo têm 300 dimensões. Se o modelo tiver vetores de uma dimensão diferente, esse número deve ser ajustado.

- for pn in palavras:: Itera sobre cada palavra na lista de palavras.
- try:: Tenta obter o vetor da palavra no modelo.
- vetor_resultante += modelo.get_vector(pn): Se a palavra estiver no vocabulário do modelo, o vetor correspondente é somado ao vetor_resultante.
- except KeyError:: Se a palavra não estiver no vocabulário do modelo, um KeyError é levantado, e o código simplesmente ignora essa palavra (pass).

In [ ]:
def combinacao_de_vetores_por_soma(palavras, modelo):

    vetor_resultante = np.zeros((1,300))

    for pn in palavras:
        try:
            vetor_resultante += modelo.get_vector(pn)

        except KeyError:
            pass


    return vetor_resultante

vetor_texto = combinacao_de_vetores_por_soma(tokens, w2v_modelo_cbow)
print(vetor_texto.shape)
print(vetor_texto)

(1, 300)
[[-0.27450757 -0.59825522  0.69133548 -1.13243518  0.18720647  2.12337512
  -1.84139195  0.74638875  0.23407404 -0.04284739  1.10862119  0.89352629
  -2.31446788 -0.47999281  0.87822711 -2.21164209 -0.25529537 -1.24711985
   0.68264387  0.89696053 -0.30993313 -1.39740703 -0.1914977   0.07912045
   0.27154632 -2.1578359   0.50183762 -1.02607948 -0.41872717  1.54443856
  -0.50024045 -1.71675737  1.23583828  2.47640713  1.589202    0.08996588
  -2.01210493 -0.42342442  0.06255623 -0.38721713 -0.63647851  1.03961195
   1.279463   -1.52828383 -2.22172838 -0.11052847  1.96253443 -0.7456958
   1.66987214 -0.15660812 -1.42984989 -1.97375983  0.82749984 -1.95111883
  -0.58955556  1.14995566  0.86756    -1.33911222  1.42421293  1.32092891
  -1.1612137  -2.22032329  1.79152638 -0.34962308  0.98687069  0.12655459
   0.41527253  0.40892041  0.11260994 -0.50725175  1.38428345  2.05934276
  -0.32172359  0.1001855   0.88106175  1.32230061  1.4537811   0.40400892
  -1.92261148  1.0899249  -1.2

##Função matriz vetores

- x é o número de textos.
- y é o tamanho dos vetores no modelo (neste caso, assumindo 300 dimensões).
- matriz é uma matriz de zeros com dimensão (x, 300).
- for i in range(x): itera sobre cada texto.
- palavras = tokenizador(textos.iloc[i]) tokeniza o texto na posição i usando a função tokenizador.
- matriz[i] = combinacao_de_vetores_por_soma(palavras, modelo) calcula a soma dos vetores das palavras no texto e armazena o resultado na i-ésima linha da matriz.

In [ ]:
def matriz_vetores(textos, modelo):
    x = len(textos)
    y = 300
    matriz = np.zeros((x,y))

    for i in range(x):
        palavras = tokenizador(textos.iloc[i])
        matriz[i] = combinacao_de_vetores_por_soma(palavras, modelo)

    return matriz

matriz_vetores_treino_cbow = matriz_vetores(artigo_treino.title, w2v_modelo_cbow)
matriz_vetores_teste_cbow = matriz_vetores(artigo_teste.title, w2v_modelo_cbow)
print(matriz_vetores_treino_cbow.shape)
print(matriz_vetores_teste_cbow.shape)

(90000, 300)
(20513, 300)


#Treinamento dos modelos

##Modelo cbow

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

def classificador(modelo, x_treino, y_treino, x_teste, y_teste):

    RL = LogisticRegression(max_iter = 800)
    RL.fit(x_treino, y_treino)
    categorias = RL.predict(x_teste)
    resultados = classification_report(y_teste, categorias)
    print(resultados)

    return RL

RL_cbow = classificador(w2v_modelo_cbow,
                        matriz_vetores_treino_cbow,
                        artigo_treino.category,
                        matriz_vetores_teste_cbow,
                        artigo_teste.category)


              precision    recall  f1-score   support

     colunas       0.80      0.71      0.75      6103
   cotidiano       0.64      0.80      0.71      1698
     esporte       0.92      0.86      0.89      4663
   ilustrada       0.13      0.86      0.23       131
     mercado       0.84      0.78      0.81      5867
       mundo       0.74      0.84      0.79      2051

    accuracy                           0.79     20513
   macro avg       0.68      0.81      0.70     20513
weighted avg       0.82      0.79      0.80     20513



##Modelo SkipGram

In [ ]:
matriz_vetores_treino_sg = matriz_vetores(artigo_treino.title, w2v_modelo_sg)
matriz_vetores_teste_sg = matriz_vetores(artigo_teste.title, w2v_modelo_sg)

RL_sg = classificador(w2v_modelo_sg,
                        matriz_vetores_treino_sg,
                        artigo_treino.category,
                        matriz_vetores_teste_sg,
                        artigo_teste.category)

              precision    recall  f1-score   support

     colunas       0.80      0.71      0.76      6103
   cotidiano       0.65      0.81      0.72      1698
     esporte       0.93      0.87      0.90      4663
   ilustrada       0.14      0.89      0.24       131
     mercado       0.84      0.79      0.82      5867
       mundo       0.75      0.84      0.79      2051

    accuracy                           0.79     20513
   macro avg       0.69      0.82      0.70     20513
weighted avg       0.82      0.79      0.80     20513



###Modelos com resultados bem parecidos.